# Project 3: Differentiate Reddit Biotech and Bioinformatics Subreddits 

Create a model to differentiate Reddit 

In [1]:
# Imports
import requests
import pandas as pd

In [2]:
# Link to the Reddit 
url = 'https://api.pushshift.io/reddit/search/submission'

## Functions

In [3]:
def get_subreddit_data(subreddit, limit, until = None):
    '''
    Input:
    Subreddit name, 
    Limit of the input, 
    Until - default None
    
    Output: the Data Frame with responce 
    '''
    params = {
        'subreddit': subreddit,
        'limit': limit,
        'filter': 'subreddit, selftext,title, created_utc',
        'until': until
    }


    res = requests.get(url, params)

    print(f'Status code {res.status_code}')
    
    return pd.DataFrame(res.json()['data'])

In [4]:
def get_data_n_times(n, subreddit, limit):
    '''
    Input:
    Number of times to get data
    Subreddit name
    Limit per request   
    
    Output Data Frame with all collected data
    '''
    data = get_subreddit_data(subreddit, limit, until = None)
    last_created_utc = data['created_utc'].tail(1)
    for _ in range(n-1):
        data = pd.concat([data, get_subreddit_data(subreddit, limit, until = last_created_utc)], ignore_index=True)
        last_created_utc = data['created_utc'].tail(1)
    return data

---

# Collecting the Data from Reddit using PushShift

In [10]:
# Collecting data from the Biology subreddit
bioinformatics = get_data_n_times(8, 'bioinformatics', 100)

Status code 200
Status code 200
Status code 200
Status code 200
Status code 200
Status code 200
Status code 200
Status code 200


In [11]:
# Checking the amount of data available 
bioinformatics.shape

(654, 4)

In [13]:
# Collecting data from the Health subreddit
biotech = get_data_n_times(8, 'biotech', 100)

Status code 200
Status code 200
Status code 200
Status code 200
Status code 200
Status code 200
Status code 200
Status code 200


In [14]:
# Checking the amount of data available 
biotech.shape

(795, 4)

In [17]:
# Conmine data from two reddits
reddit = pd.concat([bioinformatics, biotech], ignore_index=True)

In [18]:
reddit.shape

(1449, 4)

In [19]:
# Save data to file
reddit.to_csv('../data/reddit.csv', index=False)

---

I gathered about the same ammount of data for both subreddits.
Next I will clean and prepare the data for modeling